In [43]:
import pandas as pd
covid_data = pd.read_csv('data/covid_data.csv')
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]

## 3. Знакомимся с новыми данными: коронавирус


Мы будем изучать методы визуализации, используя два набора данных:

1. [covid_data](https://lms-cdn.skillfactory.ru/assets/courseware/v1/e8e401d5f46b6c4e81f7330cc24d4fe2/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/covid_data.zip) — данные о показателях заболеваемости за каждый день в различных регионах 187 стран. [(источник данных)](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)

Мы будем работать со следующими столбцами:

* *date* — дата наблюдения;
* *province*/*state* — наименование провинции/штата;
* *country* — наименование страны;
* *confirmed* — общее число зафиксированных случаев на указанный день;
* *deaths* — общее число зафиксированных смертей на указанный день;
* *recovered* — общее число выздоровлений на указанный день

2. [country_vaccinations](https://lms-cdn.skillfactory.ru/assets/courseware/v1/4f12d963943a898af014981a4a822088/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/country_vaccinations.zip) — данные о процессе вакцинирования людей в 221 стране мира. [(источник данных)](https://www.kaggle.com/gpreda/covid-world-vaccination-progress)

> Обратите внимание, что в приведённом источнике данные о вакцинации постоянно обновляются. В данном модуле будет использоваться файл, который мы скачали заранее.

Данная таблица содержит следующие столбцы:

* *country* — наименование страны;
* *date* — дата наблюдения;
* *total_vaccinations* — общее число введённых вакцин в стране на указанный день;
* *people_vaccinated* — общее число привитых первым компонентом в стране на указанный день;
* *people_vaccinated_per_hundred* — процент привитых первым компонентом в стране на указанный день (рассчитывается как *people_vaccinated / population* * 100);
* *people_fully_vaccinated* — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
* *people_fully_vaccinated_per_hundred* — процент привитых вторым компонентом в стране на указанный день (рассчитывается как *people_fully_vaccinated* / *population* * 100);
* *daily_vaccination* — ежедневная вакцинация (число вакцинированных в указанный день);
* *vaccines* — комбинации вакцин, используемые в стране.

#### НЕМНОГО ОБ ОСОБЕННОСТЯХ ДАННЫХ
В таблице *covid_data* данные о заболеваемости представлены для каждого региона (штата) страны за каждый день.    
Например, для США (United States) записи за 29 мая 2021 года имеют вид:   

![таблица](https://lms-cdn.skillfactory.ru/assets/courseware/v1/230393ba3d8d6c83c12538a8c4f1d0c6/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md13_3_4.png)

В таблице *vaccinations_data* параметры вакцинирования отображены для всех стран на каждый день, без учёта регионов.    
Вот часть таблицы для тех же Соединённых Штатов за 29 мая 2021 года. Как видите, она всего одна:

![таблица](https://lms-cdn.skillfactory.ru/assets/courseware/v1/83b7cc0f85531b6c2f07146579fab466/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md13_3_5.png)

В данных много пропусков, а также неверно введённых значений. Например, для США (United States) и Великобритании (United Kingdom) в большинстве записей таблицы covid_data число выздоровевших людей равно 0.    
В данном модуле мы закроем на это глаза, но вообще эти страны подлежали бы исключению из данных или нам понадобился бы дополнительный источник данных, из которого мы смогли бы получить недостающую информацию.

> братите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, то есть с каждым днём они должны расти.    
Такие признаки называют накопительными.

Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

1. В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.
2. В таблицах не совпадает число стран, а иногда и их названия. 
При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны).    
Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.
3. Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

#### ПРЕДОБРАБОТКА ДАННЫХ
Опираясь на замечания выше, выполним небольшую предобработку.

В таблице *covid_data*:

1. Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

In [44]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

2. Преобразуем даты в формат datetime с помощью функции **pd.to_datetime()**:

In [45]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

3. Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

In [46]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

4. Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам.   
После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода *diff*():

In [47]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

Итоговый вид таблицы *covid_data*:

![covid_data](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c372c51b2676436625da5f1402faa1f1/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md13_3_6.png)

В таблице *vaccinations_data* достаточно будет преобразовать столбцы в формат *datetime*:

In [48]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

### Practice.3

### Задание 3.1 - 3.2
За какой период представлены данные в таблице covid_data? В качестве ответа введите даты в формате datetime (без указания времени).
За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.

In [49]:
print(f"Данные по заболеваемости представленны за период с {covid_data['date'].min()} по {covid_data['date'].max()}")
print(f"Данные по вакцинации представленны за период с {vaccinations_data['date'].min()} по {vaccinations_data['date'].max()}")


Данные по заболеваемости представленны за период с 2020-01-22 00:00:00 по 2021-05-29 00:00:00
Данные по вакцинации представленны за период с 2020-12-02 00:00:00 по 2021-09-06 00:00:00


### Задание 3.3
С помощью метода *merge*() объедините таблицы *covid_data* и *vaccinations_data* по столбцам *date* и *country*.   
Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1.    
То есть в результирующую таблицу должны попасть все записи из таблицы *covid_data* и из её пересечения с *vaccinations_data*, но не более.    
Результат объединения занесите в переменную ***covid_df***.

In [51]:
covid_df = covid_data.merge(
    vaccinations_data,
    on=['date', 'country'],
    how='left'    
)

print(f"Число строк {covid_df.shape[0]}")
print(f"Число столбцов {covid_df.shape[1]}")

Число строк 86785
Число столбцов 16


В получившейся в задании 3.3 таблице *covid_df* создайте признаки *death_rate* — общий процент смертей среди зафиксированных случаев (летальность) и *recover_rate* — общий процент случаев выздоровления.   
Данные характеристики рассчитайте как отношение числа смертей (*deaths*) и числа выздоровлений (*recovered*) к числу зафиксированных случаев (*confirmed*) и умножьте результаты на 100%.

In [63]:
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed']) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed']) * 100

### Задание 3.4

Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

In [61]:
print(f"Максимальная летальность в США равна : {round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)}")

Максимальная летальность в США равна : 7.19


### Задание 3.5
Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.


In [65]:
print(f"Средний процент выздоровивших в России равен: {round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)}")

Средний процент выздоровивших в России равен: 67.06


## 4. Графические возможности библиотеки Pandas

> **Примечание**.   
Для визуализации Pandas использует библиотеку matplotlib, о которой мы поговорим в следующем юните. Если попробовать воспользоваться методами визуализации Pandas без предварительной установки matplotlib, вы получите ошибку ImportError.    
Для установки библиотеки используется стандартная команда, которую нужно выполнить в терминале:
*pip install matplotlib*
или в ячейке ноутбука:
*! pip install matplotlib*


##

### БАЗОВАЯ ВИЗУАЛИЗАЦИЯ В PANDAS

Основным методом для создания простейших графиков в Pandas является *plot*().
Основные параметры метода *plot*():
* *x* — название признака, который будет отложен по оси абсцисс;
* *y* — название признака, который будет отложен по оси ординат;
* *figsize* — размер графика (кортеж из ширины и высоты в дюймах);
* *kind* —тип визуализации. Основные типы:
* '*line*' — линейный график (по умолчанию);
* '*bar*' — столбчатая диаграмма;
* '*barh*' — горизонтальная столбчатая диаграмма;
* '*hist*' — гистограмма;
* '*box*' — коробчатая диаграмма (boxplot);
* '*pie*' — круговая диаграмма;
* '*scatter*' — диаграмма рассеяния.
* *grid* — отображение сетки (по умолчанию False);
* *legend* — отображение легенды (по умолчанию False);
* *title* — название графика;
* *color* — цвет.

Начнём с исследования заболеваемости коронавирусом во всём мире. Для этого первым делом отобразим, как менялось ежедневное число заболевших (*daily_confirmed*) во всём мире во времени.    
Далее сгруппируем таблицу по датам и подсчитаем суммарное число зафиксированных случаев по дням.

Теперь мы наконец можем построить график с помощью метода *plot*(). Будем использовать линейный график размером 12x4 (попробуйте взять другие числа, чтобы увидеть разницу).    
Подпишем график и отобразим сетку. Параметр lw (*line width*) отвечает за ширину линии для линейного графика.

In [ ]:
grouped_cases = covid_df.groupby('date')['daily_confirmed'].sum()
grouped_cases.plot(
    kind='line',
    figsize=(12, 4),
    title='Ежедневная заболеваемость во времени',
    grid = True,
    lw=3
);

##### **Примечание**.


Обратите внимание, что мы не использовали параметры *x* и *y*. Это связано с тем, что в нашем случае переменная *grouped_cases* — это объект *Series*.   
*plot*() может являться методом объекта типа DataFrame или *Series*. При типе DataFrame требуется указать *х* и *у*, либо будут построены графики всех столбцов.    
При типе *Series* в этом нет необходимости, ведь это и так всего один столбец данных (у) с индексами (х).   
При этом по оси абсцисс (х) откладываются индексы (на сгруппированных данных grouped_cases индексы — это даты), а по оси ординат (у) — значения показателей.   
*Здесь и далее в модуле точка с запятой в последней строке кода используется во избежание вывода на экран служебной информации в ноутбуке (попробуйте её убрать и посмотрите, что будет).*

*На графике выше отчётливо виден умеренный начальный рост заболеваемости, после чего наблюдается её резкое повышение в середине октября 2020 года, а в декабре 2020 года — аномальная вспышка коронавируса (зафиксировано более 1.4 млн. заболевших в день).    
Такой резкий максимум, возможно, является ошибкой в данных и требует более детального разбора. Далее заметно постепенное уменьшение числа ежедневно фиксируемых случаев и наступление второй волны в марте 2021 года.    
Наконец, начиная с мая 2021 года наблюдается очередной спад.*

*Пилообразность графика (подъёмы и спады с периодом в 7 дней) может быть связана с рабочими и выходными днями.*

*Нам неизвестно, как устроен во времени процесс постановки диагноза и сбора статистики и отличается ли он в разных странах. Можно предположить, что:*

* *спад в выходные обусловлен меньшей интенсивностью работы медицинских служб;*
* *в тех случаях, когда диагноз ставится по результатам анализа, если в выходные берётся/проводится меньше тестов, к понедельнику они ещё не готовы и диагноз ещё не поставлен;*
* *минимум в понедельник может быть запаздыванием подсчёта статистики, т.е. на самом деле данные за понедельник — это данные за воскресенье.*

Теперь построим гистограмму, которая покажет распределение ежедневной заболеваемости во всём мире.    
Для этого параметр *kind* выставляем на значение '*hist*'. Параметр *bins* (корзины) отвечает за число прямоугольников в гистограмме — пусть их будет 10 (попробуйте использовать другие числа, чтобы увидеть разницу).

In [ ]:
grouped_cases.plot(
    kind='hist',
    figsize=(10, 6),
    title='Распределение ежедневной заболеваемости',
    grid = True,
    color = 'black',
    bins=10
);

*По гистограмме можно судить о частоте попадания ежедневной заболеваемости в определённый интервал.    
На оси абсцисс отложен диапазон ежедневной заболеваемости (в млн человек), разбитый на десять равных интервалов, на оси ординат — число наблюдений, попавших в этот интервал.*

1. *Мы видим модальное значение около нуля (от 0 до примерно 150 тыс. заболеваний в день), то есть большинство наблюдений сосредоточено в первом интервале,    
далее частота постепенно падает. Это связано с тем, что долгое время распространение вируса было довольно слабым.*
2. *В глаза бросается «пенёк», соответствующий резкой вспышке заболеваемости, которую мы видели ранее. Его высота очень мала, ведь такое наблюдение единственное.    
Даже на гистограмме кажется, что данное наблюдение является аномальным и, скорее всего, представляет собой выброс.*

Давайте построим столбчатую диаграмму, которая покажет ТОП-10 стран по суммарной заболеваемости.

Для этого сгруппируем данные по странам и вычислим последний зафиксированный показатель с помощью агрегирующего метода [last()](https://pandas.pydata.org/pandas-docs/version/1.5/reference/api/pandas.core.groupby.GroupBy.last.html) — он возвращает последнее значение в столбце DataFrame.

Для построения столбчатой диаграммы значение параметра kind выставляем на 'bar'. Параметр colormap отвечает за [цветовую гамму](https://matplotlib.org/stable/tutorials/colors/colormaps.html) графика.

In [ ]:
grouped_country = covid_df.groupby(['country'])['confirmed'].last()
grouped_country = grouped_country.nlargest(10)
grouped_country.plot(
    kind='bar',
    grid=True,
    figsize=(12, 4),
    colormap='plasma'
);

*На диаграмме видно, что лидерами по числу заболевших являются Соединённые Штаты, Индия и Бразилия.    
Соотношение зафиксированных случаев заражения в остальных странах примерно одинаковое.*

А теперь посмотрим, как в этих десяти странах соотносится количество заболевших и умерших от вируса.    
Для этого отобразим сразу два показателя на столбчатой диаграмме: 

In [ ]:
grouped_country = covid_df.groupby(['country'])[['confirmed', 'deaths']].last()
grouped_country = grouped_country.nlargest(10, columns=['confirmed'])
grouped_country.plot(     kind='bar',     grid=True,     figsize=(12, 4), );

*Этот график является небольшим усовершенствованием предыдущего. Теперь на нём можно увидеть соотношение зафиксированных случаев заражения и смертей.*

*Очевидно, что отношение числа умерших к числу заболевших весьма низкое. Также это может косвенно говорить о разных методиках учёта заболевших (например, какие-то страны могут учитывать заболевших только по мазку,    
в то время как другие — по клинической картине, учитывающей и другие показатели). То есть с методической точки зрения учёт по числу заражений может быть не совсем корректным.    
Лучшим показателем будет являться число смертей (хотя и этот метод не идеален).*

*Более того, если построить график с сортировкой не по числу заболевших, а по числу умерших, поменяются и места, и страны в рейтинге. Попробуйте построить такой график сами!*

> Визуализация с помощью Pandas является удобным инструментом, когда графики необходимо построить «здесь и сейчас», не сильно заботясь об их внешнем виде. Однако такой подход имеет значительный минус по сравнению с использованием специализированных библиотек для визуализации — довольно ограниченный функционал:

* С помощью Pandas можно построить лишь базовый набор диаграмм. Для построения более сложных видов визуализации Pandas не подходит.
* Трудно настроить визуализацию нескольких видов графиков одновременно в разных масштабах.
* Сложно (иногда и вовсе невозможно) корректировать внешний вид графика.
* Отсутствует 3D-визуализация.